# PINNs para el sistema masa–resorte–amortiguador

**Autor:** David Ortiz — 2025

Accede al trabajo fundacional de las PINNs [aquí](https://www.sciencedirect.com/science/article/pii/S0021999118307125).

Este tutorial está inspirado en [este workshop](https://github.com/benmoseley/harmonic-oscillator-pinn-workshop), desarrollado por Ben Moseley.

### Introducción
Las Redes Neuronales Informadas por la Física (PINNs) integran ecuaciones gobernantes directamente en la función de pérdida mediante diferenciación automática. Este enfoque reduce la dependencia de grandes volúmenes de datos y aporta coherencia física a la solución aprendida.  
En esta actividad se aplican PINNs al modelo lineal clásico **masa–resorte–amortiguador**, para el cual existe una solución analítica conocida, lo que permite evaluar cuantitativamente el desempeño del método.

### Objetivos de aprendizaje
- Implementar una PINN desde cero en PyTorch para resolver el sistema masa–resorte–amortiguador siguiendo un esquema estructurado de seis etapas.  
- Identificar y comprender los distintos términos de la función de pérdida de una PINN, incluyendo el residuo de la EDO y las condiciones iniciales.  
- Analizar el proceso de entrenamiento de una PINN y el uso de diferenciación automática para calcular derivadas de la red neuronal.

### Resumen de la actividad
Se construye una PINN para el sistema masa–resorte–amortiguador siguiendo los seis pasos introducidos en el taller teórico:

<img src="../figures/pinns_new_scheme.png" width="800" height="400">

- (1) formular el modelo matemático;  
- (2) definir el dominio del problema;  
- (3) implementar una ANN como aproximador de la solución $x(t)$;  
- (4) emplear diferenciación automática para obtener derivadas;  
- (5) diseñar la función de pérdida con términos físicos (residuo de la EDO y condición inicial);  
- (6) seleccionar y configurar un optimizador (Adam).

Finalmente, se ejecuta el ciclo de entrenamiento y se comparan los resultados obtenidos con la solución exacta.


## 0. Importamos funciones 

In [ ]:
%matplotlib inline

In [ ]:
# Import NumPy for numerical operations
import numpy as np
# Import PyTorch for building and training neural networks
import torch
import torch.nn as nn
import torch.optim as optim
# Import Matplotlib for plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
# Import the time module to time our training process
import time
# Ignore Warning Messages
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Setup (device + plots)
def get_device() -> str:
    return "cuda" if torch.cuda.is_available() else \
           "mps"  if torch.backends.mps.is_available() else "cpu"

def set_mpl_style(gray: str = "#5c5c5c") -> None:
    mpl.rcParams.update({
        "image.cmap": "viridis",
        "text.color": gray, "xtick.color": gray, "ytick.color": gray,
        "axes.labelcolor": gray, "axes.edgecolor": gray,
        "axes.spines.right": False, "axes.spines.top": False,
        "axes.formatter.use_mathtext": True, "axes.unicode_minus": False,
        "font.size": 15, "interactive": False, "font.family": "sans-serif",
        "legend.loc": "best", "text.usetex": False, "mathtext.fontset": "stix",
    })

device = get_device()
print(f"Using {device} device")
set_mpl_style()

# Metrics
def relative_l2_error(u_num: torch.Tensor, u_ref: torch.Tensor) -> torch.Tensor:
    return torch.norm(u_num - u_ref) / torch.norm(u_ref)

# Util function to plot the solutions
def plot_comparison(t, theta_true, theta_pred, loss):
    t, u, u_hat = (
        x.detach().cpu().numpy().ravel()
        for x in (t, theta_true, theta_pred)
    )

    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    ax[0].plot(t, u, label=r'$\theta(t)$ (numerical)')
    ax[0].plot(t, u_hat, label=r'$\theta_{\mathrm{pred}}(t)$')
    ax[0].set(title='Numerical vs Predicted',
              xlabel=r'Time $(s)$', ylabel='Amplitude', ylim=(-1, 1.3))
    ax[0].legend(frameon=False)

    ax[1].plot(t, np.abs(u - u_hat))
    ax[1].set(title='Absolute Difference',
              xlabel=r'Time $(s)$',
              ylabel=r'$|\theta - \theta_{\mathrm{pred}}|$')

    fig.tight_layout()
    plt.show()

    fig, ax = plt.subplots(figsize=(6, 3))
    ax.plot(loss)
    ax.set(title='Training Progress',
           xlabel='Iteration', ylabel='Loss',
           xscale='log', yscale='log')
    ax.grid(True)
    fig.tight_layout()
    plt.show()

### 1. Modelo matemático del sistema masa–resorte–amortiguador

El sistema masa–resorte–amortiguador describe el movimiento de una masa $m$ unida a un resorte de constante elástica $k$ y a un elemento disipativo con coeficiente de amortiguamiento $c$. En ausencia de fuerzas externas, su ecuación de movimiento está dada por:

\begin{equation*}
m\,\ddot{x}(t) + c\,\dot{x}(t) + k\,x(t) = 0
\end{equation*}

donde $x(t)$ representa el desplazamiento de la masa respecto a su posición de equilibrio, $\dot{x}(t)$ su velocidad y $\ddot{x}(t)$ su aceleración.

Dividiendo la ecuación por la masa $m$, se obtiene una forma normalizada que resulta más conveniente para el análisis:

\begin{equation*}
\ddot{x}(t) + 2\zeta\omega_n\,\dot{x}(t) + \omega_n^2\,x(t) = 0
\end{equation*}

donde se han introducido los parámetros estándar del sistema:

\begin{equation*}
\omega_n = \sqrt{\frac{k}{m}} \qquad \text{frecuencia natural no amortiguada}
\end{equation*}

\begin{equation*}
\zeta = \frac{c}{2\sqrt{km}} \qquad \text{razón de amortiguamiento adimensional}
\end{equation*}

En lo que sigue, trabajaremos con esta formulación normalizada y fijaremos valores específicos para los parámetros del sistema.


In [ ]:
# Dominio temporal
T = 5.0        # tiempo total de simulación
x0 = 1.0       # Posición inicial 
v0 = 0.0       # velocidad incial
wn = 5.0       # Frecuencia natural
zeta = 0.2     # razón de amortiguamiento

### 2. Definición del dominio temporal

En esta etapa se define el dominio sobre el cual la PINN será **entrenada** y **evaluada**. El tiempo $t$ se considera en el intervalo $[0, T]$ y se muestrea en un conjunto de puntos que actuarán como entradas de la red neuronal durante el entrenamiento.

> **💡 Nota**  
> Cada punto $t_i \in [0, T]$, con $i = 1, \dots, N_{\text{sample}}$, se denomina **punto de colocación**.  
> Este término proviene de la estrecha relación entre las PINNs y los métodos numéricos de colocación clásicos.

> **💡 Nota**  
> Para evaluar correctamente el desempeño del modelo, es conveniente utilizar conjuntos de puntos distintos para entrenamiento y evaluación.


In [ ]:
def crear_dominio_temporal(T, N_train=101, N_eval=1000):
    """Crea el dominio temporal para la PINN."""
    t_train = torch.linspace(0, T, N_train, 
                             device=device, 
                             requires_grad=True).view(-1, 1)  # entrenamiento
    t_eval = torch.linspace(0, T, N_eval, 
                             device=device, 
                             requires_grad=True).view(-1, 1)  # evaluación
    return t_train, t_eval # dominio de evaluación

### 3. Red neuronal como aproximador de la solución

En esta etapa se define una red neuronal que actúa como un aproximador paramétrico de la solución de la ecuación diferencial. La función desconocida $x(t)$ se reemplaza por una red neuronal dependiente de un conjunto de parámetros $\Theta$:

$$
x_{\text{PINN}}(t; \Theta) \approx x_{\text{real}}(t).
$$

Los parámetros del modelo corresponden a los pesos y sesgos de cada capa de la red, que se agrupan como:

$$
\Theta = \{ W_i, b_i \}_{i=1}^{L},
$$

donde $W_i$ y $b_i$ representan los pesos y sesgos de la capa $i$, respectivamente, y $L$ es el número total de capas. Estos parámetros se ajustan durante el entrenamiento con el objetivo de minimizar el error asociado al cumplimiento de la ecuación física.

> **💡 Nota**  
> En este ejercicio se utiliza un perceptrón multicapa (MLP) con funciones de activación *tanh* y una inicialización de pesos basada en el esquema de Glorot. El número de capas ocultas, el número de neuronas por capa y la función de activación se denominan **hiperparámetros** de la red.

In [ ]:
# Define a neural network class with user defined layers and neurons
class NeuralNetwork(nn.Module):

    def __init__(self, hlayers = [1, 10, 10, 1]):
        super(NeuralNetwork, self).__init__()

        layers = []
        for i in range(len(hlayers[:-2])):
            layers.append(nn.Linear(hlayers[i], hlayers[i+1]))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(hlayers[-2], hlayers[-1]))

        self.layers = nn.Sequential(*layers)
        self.init_params()

    def init_params(self):
        """Xavier Glorot parameter initialization of the Neural Network
        """
        def init_normal(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight) # Xavier
        self.apply(init_normal)

    def forward(self, x):
        return self.layers(x)

### 4. Diferenciación automática en PyTorch

Antes de continuar, es necesario establecer cómo calcular derivadas de la salida de una red neuronal respecto a sus entradas. Para ello se utiliza la **diferenciación automática** (*automatic differentiation* o *autodiff*), una técnica que permite evaluar gradientes de forma eficiente y exacta aplicando sistemáticamente la regla de la cadena.

En PyTorch, las operaciones realizadas sobre tensores se registran en un **grafo computacional dinámico**. Este gráfico permite calcular derivadas de manera automática mediante un proceso de propagación hacia atrás (*backpropagation*), lo que resulta fundamental para imponer ecuaciones diferenciales directamente sobre la salida de la red.


In [ ]:
# Util function to calculate tensor gradients with autodiff
def grad(outputs, inputs):
    """Computes the partial derivative of an output with respect
    to an input.
    Args:
        outputs: (N, 1) tensor
        inputs: (N, D) tensor
    """
    return torch.autograd.grad(outputs, inputs,
                        grad_outputs=torch.ones_like(outputs),
                        create_graph=True,
                        )[0]


# Definir tensor de entrada. Si queremos derivar c/r a x necesitamos inicializar con requires_grad=True
x = torch.tensor([1.0, 2.0, 3.0], device=device, requires_grad=True).view(-1,1).float() # (N,1)

# Calcular operación que dependen de x
y = x**2 # (N,1)  

# Calcular derivadas c/r a x 
# grad es un wrapper de torch.autograd
dy_dx = grad(y, x) 

# Calcular derivadas de orden superior
d2y_dx2 = grad(dy_dx, x)  

print("x:", x)
print("y = x^2:", y)
print("dy/dx:", dy_dx)
print("d^2y/dx^2:", d2y_dx2)  

# Esto también funciona para redes neuronales
# test_ANN = NeuralNetwork()

# NNx = test_ANN(x)
# dNNx_dx = grad(NNx, x)
# print("NNx: ", dNNx_dx)
# print("dNNx/dx:", dNNx_dx)

### 5. Función de pérdida informada por la física

El entrenamiento de la PINN se basa en el modelo normalizado del sistema masa–resorte–amortiguador, caracterizado por la frecuencia natural $\omega_n$ y la razón de amortiguamiento $\zeta$:

$$
\ddot{x}(t) + 2\zeta\omega_n\,\dot{x}(t) + \omega_n^2 x(t) = 0.
$$

La solución desconocida $x(t)$ se aproxima mediante la salida de la red neuronal $x_{\text{PINN}}(t;\Theta)$. A partir de esta aproximación se define el **residuo físico** de la ecuación diferencial y las **condiciones iniciales**:

$$
\begin{aligned}
f_{\text{ode}}(t;\,x_{\text{PINN}}) &=
\frac{d^2 x_{\text{PINN}}}{dt^2}
+ 2\zeta\omega_n \frac{d x_{\text{PINN}}}{dt}
+ \omega_n^2 x_{\text{PINN}}, \\
g_{\text{ic}}(0;\,x_{\text{PINN}}) &=
x_{\text{PINN}}(0;\Theta) - x_0, \\
h_{\text{ic}}(0;\,x_{\text{PINN}}) &=
\frac{d x_{\text{PINN}}(0;\Theta)}{dt} - v_0.
\end{aligned}
$$

La función de pérdida total se construye como una combinación ponderada de estos términos:

$$
\mathcal{L}(\Theta) =
\frac{\lambda_1}{N}\sum_i f_{\text{ode}}(t_i;\,x_{\text{PINN}})^2
+ \lambda_2\, g_{\text{ic}}(0;\,x_{\text{PINN}})^2
+ \lambda_3\, h_{\text{ic}}(0;\,x_{\text{PINN}})^2.
$$

El entrenamiento de la PINN consiste en resolver el problema de optimización

$$
\min_{\Theta} \; \mathcal{L}(\Theta),
$$

utilizando diferenciación automática (`torch.autograd`) para evaluar las derivadas temporales de la red necesarias para el cálculo del residuo físico.

> **💡 Nota**  
> Si la función de pérdida incluye únicamente términos físicos, el esquema es *data-free*.  
> Al incorporar términos asociados a datos observados, el enfoque pasa a ser *data-driven*.

> **💡 Nota**  
> En este marco, las condiciones iniciales (y de frontera, cuando existen) se imponen de manera débil a través de la función de pérdida.


In [ ]:
# Define a loss function (Mean Squared Error) for training the network
MSE_func = nn.MSELoss()

# derivatives of the ANN
def PINNLoss(PINN, t_phys, wn, zeta, x0 = 1, v0 = 0,
             lambda1 = 1, lambda2 = 1, lambda3 = 1):

    t0 = torch.tensor(0., device=device, requires_grad=True).view(-1,1)

    # ANN output, first and second derivatives
    x_pinn_t = PINN(t_phys)
    x_pinn_dt = grad(x_pinn_t, t_phys)
    x_pinn_ddt = grad(x_pinn_dt, t_phys)
    
    f_ode = x_pinn_ddt + 2 * zeta * wn * x_pinn_dt + wn**2 * x_pinn_t
    ODE_loss = lambda1 * MSE_func(f_ode, torch.zeros_like(f_ode)) 
    
    g_ic = PINN(t0)
    IC_loss = lambda2 * MSE_func(g_ic, torch.ones_like(g_ic)*x0)
    
    h_bc = grad(PINN(t0),t0)
    BC_loss = lambda3 * MSE_func(h_bc, torch.ones_like(h_bc)*v0)
    
    return ODE_loss + IC_loss + BC_loss 

### 6. Definición del optimizador

Una vez definida la función de pérdida, se selecciona un optimizador para ajustar los parámetros de la red neuronal $\Theta = \{W_i, b_i\}$. El objetivo es minimizar la pérdida informada por la física mediante un esquema de optimización iterativo:

$$
\min_{\Theta} \mathcal{L}(\Theta),
\qquad
\Theta^{k+1} = \Theta^{k} - \alpha \nabla_{\Theta} \mathcal{L}(\Theta^{k}).
$$

En esta etapa se utiliza el optimizador **Adam**, un método de descenso de gradiente adaptativo que ajusta dinámicamente la tasa de aprendizaje de cada parámetro. Su combinación de momento y escalado adaptativo de gradientes lo hace especialmente adecuado para el entrenamiento estable de PINNs.

> **💡 Nota**  
> 

In [ ]:
def pinn_optimizer(pinn, lr = 0.01):

    # Define an optimizer (Adam) for training the network
    return optim.Adam(pinn.parameters(), lr=lr,
                        betas= (0.99,0.999), eps = 1e-8)

### Ciclo de entrenamiento

En esta etapa se ejecuta el proceso iterativo mediante el cual la PINN ajusta sus parámetros $\Theta = \{W_i, b_i\}$ para minimizar la función de pérdida definida previamente. En cada época (*epoch*), el modelo evalúa el residuo de la ecuación diferencial y las condiciones iniciales en los puntos de colocación $t_i \in [0, T]$, y actualiza los parámetros según el gradiente de la pérdida:

$$
\Theta \leftarrow \Theta - \eta \, \nabla_\Theta \mathcal{L}(\Theta).
$$

El entrenamiento continúa hasta que la pérdida converge o deja de disminuir de forma apreciable. Una vez finalizado, la solución aproximada $x_{\text{PINN}}(t;\Theta)$ se compara con la solución exacta para evaluar la calidad del ajuste y la capacidad del modelo de capturar la dinámica del sistema masa–resorte–amortiguador.


A continuación se presenta el código completo:

In [ ]:
#===============================================================================
# ETAPA 1: INFORMACIÓN DEL MODELO FÍSICO
#===============================================================================
# Dominio temporal
T = 5.0        # tiempo total de simulación
x0 = 1.0       # Posición inicial 
v0 = 0.0       # velocidad incial
wn = 5.0  # Frecuencia natural
zeta = 0.2     # razón de amortiguamiento

#===============================================================================
# ETAPA 2: DEFINICIÓN DEL DOMINIO 
#===============================================================================
# Creamos los tensores de tiempo para el entrenamiento y la evaluación
t_train, t_eval = crear_dominio_temporal(T)

#===============================================================================
# ETAPA 3: CREACIÓN DE LA RED NEURONAL SURROGANTE 
#===============================================================================
# Creamos la ANN
torch.manual_seed(123)
hidden_layers = [1, 30, 30, 30, 1]# Parámetros de la 

# Create an instance of the neural network
x_pinn = NeuralNetwork(hidden_layers).to(device)
nparams = sum(p.numel() for p in x_pinn.parameters() if p.requires_grad)
print(f'Number of trainable parameters: {nparams}')

#==========================================================================
# ETAPA 4 Y 5: DEFINICIÓN DE LA FUNCIÓN DE COSTO BASADA EN LA FÍSICA
#==========================================================================
# Define a loss function (Mean Squared Error) for training the network
MSE_func = nn.MSELoss()

# derivatives of the ANN
def PINNLoss(PINN, t_phys, wn, zeta, x0 = 1, v0 = 0, 
             w1 = 1, w2 = 1, w3 = 1):

    t0 = torch.tensor(0., device=device, requires_grad=True).view(-1,1)

    # ANN output, first and second derivatives
    x_pinn_t = PINN(t_phys)
    x_pinn_dt = grad(x_pinn_t, t_phys)
    x_pinn_ddt = grad(x_pinn_dt, t_phys)
    
    f_ode = x_pinn_ddt + 2 * zeta * wn * x_pinn_dt + wn**2 * x_pinn_t
    ODE_loss = w1 * MSE_func(f_ode, torch.zeros_like(f_ode)) 
    
    g_ic = PINN(t0)
    IC_loss = w2 * MSE_func(g_ic, torch.ones_like(g_ic)*x0)
    
    h_bc = grad(PINN(t0),t0)
    BC_loss = w3 * MSE_func(h_bc, torch.zeros_like(h_bc)*v0)
    
    return ODE_loss + IC_loss + BC_loss 

#==========================================================================
# ETAPA 6: DEFINICIÓN DEl OPTIMIZADOR
#==========================================================================
learning_rate = 0.01
optimizer = pinn_optimizer(x_pinn, learning_rate)

#==========================================================================
# CICLO DE ENTRENAMIENTO
#==========================================================================
training_iter = 20000

# Initialize a list to store the loss values
loss_values_pinn = []

# Start the timer
start_time = time.time()

# Training the neural network
for i in range(training_iter):

    optimizer.zero_grad()   # clear gradients for next train

    # input x and predict based on x
    loss = PINNLoss(x_pinn, t_train, wn, zeta)

    # Append the current loss value to the list
    loss_values_pinn.append(loss.item())

    if i % 500 == 0:  # print every 100 iterations
        print(f"Iteration {i}: Loss {loss.item()}")

    loss.backward() # compute gradients (backpropagation)
    optimizer.step() # update the ANN weigths

# Stop the timer and calculate the elapsed time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

## Validación

El comportamiento del sistema depende de $\zeta$:

- **Subamortiguado** ($0 < \zeta < 1$): el sistema oscila con frecuencia amortiguada  
  \begin{equation*}
  \omega_d = \omega_n \sqrt{1 - \zeta^2}
  \end{equation*}

  la solución puede escribirse en términos de las condiciones iniciales $x(0)=x_0$ y $\dot{x}(0)=v_0$:
  \begin{equation*}
  x(t) = e^{-\zeta \omega_n t}
  \left[
  x_0 \cos(\omega_d t) +
  \frac{v_0 + \zeta \omega_n x_0}{\omega_d}
  \sin(\omega_d t)
  \right]
  \end{equation*}

Esta expresión servirá como referencia para validar la solución obtenida mediante la red neuronal informada por la física.


In [ ]:
def masa_resorte_general(t, x0, v0, omega_n, zeta):
    """
    Solución exacta x(t) del sistema masa-resorte-amortiguador:
        x'' + 2*zeta*omega_n*x' + omega_n^2*x = 0
    Incluye los tres regímenes (sub, crítico y sobreamortiguado).
    """
    t = np.array(t, dtype=float)

    if 0 < zeta < 1:  # Subamortiguado
        omega_d = omega_n * np.sqrt(1 - zeta**2)
        x = np.exp(-zeta * omega_n * t) * (
            x0 * np.cos(omega_d * t) +
            (v0 + zeta * omega_n * x0) / omega_d * np.sin(omega_d * t)
        )

    else:
        raise ValueError("zeta debe ser mayor que 0.")

    return torch.tensor(x, dtype=torch.float32, device=device).view(-1, 1)

# -------------------------------------------
# solucion exacta
t_eval_np = t_eval.detach().cpu().numpy().ravel()
x = masa_resorte_general(t_eval_np, x0, v0, wn, zeta)

# predicción de la PINN
x_pred_pinn = x_pinn(t_eval)

print(f'Relative error: {relative_l2_error(x_pred_pinn, x)}')

plot_comparison(t_eval, x, x_pred_pinn, loss_values_pinn)


## **Ejercicios**:
1. Ajusta los valores de los parámetros del sistema dinámico y analiza su impacto en la convergencia de la PINN.
2. Ajusta los valores de los parámetros `lambdas` en la función de pérdida para ambas redes y analiza su impacto.
2. Modifica la tasa de aprendizaje (`learning_rate`) del optimizador y el número de iteraciones de entrenamiento, y evalúa el efecto en el desempeño.
3. Cambia el número de capas ocultas (`hidden_layers`), neuronas y funciones de activación de la red neuronal, y observa el impacto en los resultados.

## **Preguntas**:

1. **¿Cómo se puede abordar el sobreajuste en las PINNs si el objetivo es aprender los operadores subyacentes del sistema físico?**  
   <details>
   <summary>Respuesta</summary>
   El sobreajuste en las PINNs (Physics-Informed Neural Networks) es un tema ampliamente discutido. En el aprendizaje automático tradicional, el sobreajuste se asocia con la incapacidad de un modelo para generalizar a datos no vistos previamente, lo que afecta su capacidad para realizar predicciones precisas en nuevas entradas. Sin embargo, en el contexto de las PINNs, buscamos un comportamiento diferente: que la red neuronal funcione como un modelo surrogate de la solución de las ecuaciones diferenciales que describen el sistema físico. En este caso, el sobreajuste no necesariamente es perjudicial, ya que queremos que la red reproduzca fielmente la solución dentro del dominio especificado.

   El desafío relacionado con la generalización en las PINNs surge cuando estas redes se evalúan en geometrías más complejas o diferentes de las que fueron utilizadas durante el entrenamiento. En tales casos, una PINN que esté sobreajustada a una única geometría podría fallar al adaptarse a las nuevas configuraciones, comprometiendo su capacidad para generalizar y capturar las dinámicas físicas subyacentes en contextos más diversos. Por ello, abordar el sobreajuste implica equilibrar la fidelidad al dominio original y la adaptabilidad a nuevas geometrías o condiciones.
   </details>


2. **Qué ventajas ofrece el uso del MSE en la función de pérdida, dado que este enfoque puede subestimar la solución al no incorporar formulaciones integrales o variacionales**  
   <details>
   <summary>Respuesta</summary>
   El uso del MSE como función de pérdida proporciona una forma sencilla y computacionalmente eficiente de medir las diferencias puntuales entre las predicciones del modelo y los valores objetivo. Sin embargo, el MSE se centra únicamente en puntos individuales, lo que puede limitar su capacidad para capturar el comportamiento global de la solución, especialmente en dominios complejos. Al incorporar formulaciones integrales o variacionales, la función de pérdida puede reflejar el comportamiento de la solución en todo el dominio, mejorando potencialmente la precisión y estabilidad. A pesar de estas limitaciones, el MSE sigue siendo popular porque simplifica la implementación y reduce los costos computacionales, lo que lo hace adecuado para muchas aplicaciones prácticas.
   </details>



